In [1]:
import datetime
import time

import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from src.huge_dataset_project.InputLoader import InputFile
from src.huge_dataset_project.PickLoader import PickFile
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound.sound_model import HomogeneousSoundModel
from utils.physics.sound.sound_velocity_grid import MonthlySoundVelocityGrid, MonthlySoundVelocityGridOptimized

In [2]:
picks = PickFile("../../huge_dataset_project/swarm2.pick")
input = InputFile("../../huge_dataset_project/Swarm2_data.input")
stations = StationsCatalog("/home/plerolland/Bureau/dataset.yaml").filter_out_undated().filter_out_unlocated()

acoustic_grid_interp = MonthlySoundVelocityGrid([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)], interpolate=True)  # 3001.15s, mean error 0.030676462
acoustic_grid_interp_optimized = MonthlySoundVelocityGridOptimized([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)], interpolate=True)  # 1507.42s, mean error 0.030739762
# with x0.997 : mean error 0.009412665
# with x0.996 : mean error 0.00859406
# with x0.995 : mean error 0.011902887
# with x0.993 : mean error 0.024033984
# with x0.992 : mean error 0.03129123
# with x0.99 : mean error 0.046128847
acoustic_grid = MonthlySoundVelocityGrid([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)])  # 902.45s, mean error 0.043251827
acoustic_grid_optimized = MonthlySoundVelocityGridOptimized([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)])  # 361.23 s, mean error 0.043365657
acoustic_homogeneous = HomogeneousSoundModel(sound_speed=1480) # 214.73s, mean error 0.05188065

In [13]:
from scipy import signal
import geopy.distance

TIME_EXTEND = datetime.timedelta(seconds=5)
DELTA = datetime.timedelta(seconds=5)

def process_data(data, spm_min=0, spm_max=120, fs=240):
    psd = signal.welch(data, fs=fs, nperseg=256, window="hann", scaling="spectrum")
    spm_min = int(spm_min * len(psd[1]) / (fs/(2)))
    spm_max = int(spm_max * len(psd[1]) / (fs/(2)))
    psd = psd[1][spm_min:spm_max]
    return psd

RL_computed, RL_seas = [], []
SL_computed, SL_seas = [], []
for pick in tqdm(picks.picks):
    h_letters = pick.hydrophones
    h_names = [input.hydrophone_names[list(input.hydrophone_letters).index(h_letter)] for h_letter in h_letters]
    h_names = [h_name+"-bot" if h_name=="SWAMS" else h_name for h_name in h_names]
    available_s = [stations.by_date(pick.detection_times[i]).by_names(h_names[i])[0] for i in range(len(h_names))]
    
    dates = pick.detection_times
    RL_seas.append(pick.received_levels)
    SL_seas.append(pick.est_source_level)
    
    RL_computed.append([])
    SL_per_s = []
    for i in range(len(available_s)):
        manager = available_s[i].get_manager()
        manager.cache_size = 0
        data = manager.getSegment(dates[i]-TIME_EXTEND, dates[i]+TIME_EXTEND)
        new_date = dates[i] - TIME_EXTEND + datetime.timedelta(seconds=int(np.argmax(data)/240))
        data = process_data(manager.getSegment(new_date-DELTA, new_date+DELTA), spm_min=5, spm_max=60, fs=240)
        RL_computed[-1].append(10*np.log10(np.sum(data)))
        
        p = geopy.distance.geodesic(available_s[i].get_pos(), pick.est_source_pos).m
        t = 5000
        p = t*p
        
        SL_per_s.append(RL_computed[-1][-1] + 10*np.log10(p))
    SL_computed.append(np.log10(np.mean(10**np.array(SL_per_s))))

  0%|          | 1/4927 [00:13<18:14:03, 13.33s/it]


KeyboardInterrupt: 

In [5]:
to_plot = np.sqrt(np.square(errors[0,:,0].astype(np.float32)) + np.square(errors[0,:,1].astype(np.float32)))
print(np.mean(to_plot))

to_plot = np.delete(to_plot, np.abs(to_plot) > np.quantile(np.abs(to_plot), 0.95))
plt.hist(to_plot, bins=10)

KeyboardInterrupt: 

In [ ]:
x = np.array([np.mean(x.fun) for x in x_list[0]])
plt.plot(x)